In [1]:
# Boilerplate code
!apt-get update

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-s

In [2]:
# Boilerplate code
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-10 15:15:11--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.75MB/s    in 0.2s    

2020-08-10 15:15:11 (4.75 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Boilerplate code
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BDETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

raw_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
raw_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars|Love this, excell...|2015-08-31 00:00:00|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...| 

In [5]:
raw_df.count()

5115666

In [6]:
breviews = raw_df.select(["customer_id", "review_id", "product_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_body"])
breviews.show(5)

+-----------+--------------+----------+-----------+-------------+-----------+----+-----------------+--------------------+
|customer_id|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         review_body|
+-----------+--------------+----------+-----------+-------------+-----------+----+-----------------+--------------------+
|    1797882|R3I2DHQBR577SS|B001ANOOOE|          5|            0|          0|   N|                Y|Love this, excell...|
|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|          5|            0|          0|   N|                Y|The great thing a...|
|   19242472|R3LIDG2Q4LJBAO|B00HU6UQAG|          5|            0|          0|   N|                Y|Great Product, I'...|
|   19551372|R3KSZHPAEVPEAL|B002HWS7RM|          5|            0|          0|   N|                Y|I use them as sho...|
|   14802407| RAI2OIG50KZ43|B00SM99KWU|          5|            0|          0|   N|                Y|This is my go-to ...|
+-----------+-----------

In [7]:
breviews.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_body: string (nullable = true)



In [15]:
brev = breviews.dropDuplicates(['customer_id', 'product_id'])
brev.show(5)

+-----------+--------------+----------+-----------+-------------+-----------+----+-----------------+--------------------+
|customer_id|     review_id|product_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         review_body|
+-----------+--------------+----------+-----------+-------------+-----------+----+-----------------+--------------------+
|      11335|R3SS9RS87QQC9O|B00GYAXLAG|          5|            0|          0|   N|                Y|Everything came a...|
|      12719|R29R69NLB0FNT7|B00A3HR7BA|          3|            1|          1|   N|                Y|These are really ...|
|      13356| R8FOIXNS7T93Q|B000JL7WQK|          1|           11|         13|   N|                Y|I have been weari...|
|      18203|R34D0C6XEFSFNM|B00O5Q8NOA|          5|            0|          1|   N|                Y|Great product if ...|
|      21007| RXELOR363J135|B0013NFYY0|          3|            0|          0|   N|                Y|Did nothing for m...|
+-----------+-----------

In [16]:
brev_clean = brev[brev.star_rating.isNotNull()]


In [18]:
brev_clean.count()

5114925

In [20]:
brev_fin = brev_clean[brev_clean.verified_purchase.isNotNull()]
brev_fin.count()

5114924

In [21]:
# removed user and password for submission
mode = "append"
dburl = "jdbc:postgresql://<ENDPOINT>:5432/bigdatachallenge"
config = {"user":"",
          "password":"",
          "driver":"org.postgresql.Driver"}

In [24]:
brev_fin.write.jdbc(url=dburl, table='beauty_reviews', mode=mode, properties=config)